In [ ]:
pip install -q mediapipe
wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

: 

In [2]:
!wget -q -O image.jpg https://storage.googleapis.com/mediapipe-tasks/hand_landmarker/woman_hands.jpg

import cv2
# from google.colab.patches import cv2_imshow


img = cv2.imread("image.jpg")
cv2.imshow('img',img)
cv2.waitKey(0)
# cv2_imshow(img)

-1

In [5]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("image.jpg")

# STEP 4: Detect hand landmarks from the input image.
detection_result = detector.detect(image)

# STEP 5: Process the classification result. In this case, visualize it.
# annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
# cv2.imshow('img',annotated_image)
# cv2.waitKey(0)

NameError: name 'audio_classifier' is not defined

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

import joblib

# Simpan scaler

def ubah_ke_fitur_rel(df):
    kolom_xyz = [col for col in df.columns if col != "label"]
    data = df[kolom_xyz].values.reshape(-1, 21, 3)  # 21 titik, 3 dimensi

    fitur_baru = []
    for baris in data:
        anchor = baris[0]  # titik 0
        rel = baris[1:] - anchor  # titik 1-20 relatif ke titik 0
        fitur_baris = rel.flatten()  # jadi 60 elemen
        fitur_baru.append(fitur_baris)

    return np.array(fitur_baru)


# 1. Load Data
df = pd.read_csv(r"C:\Users\Aldan\Desktop\ASLClassification\Code\hand_features.csv")  # Ganti dengan nama file kamu

# 2. Pisahkan fitur dan label
X = ubah_ke_fitur_rel(df)
y = df["label"].values

# 3. Encode label
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded, num_classes=len(df["label"].unique()))

# 4. Normalisasi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler.pkl")

# 5. Split data ke training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

# 6. Buat Neural Network
model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(36, activation='softmax')  # 36 kelas
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 7. Latih model
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2)

# 8. Evaluasi
loss, acc = model.evaluate(X_test, y_test)
print(f"Akurasi: {acc*100:.2f}%")
model.save("Model_ASL.h5")


import tensorflow as tf

# 1. Load model HDF5
# model = tf.keras.models.load_model(r"C:\Users\Aldan\Desktop\ASLClassification\Code\Model_ASL.h5")

# 2. Konversi ke TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 3. Simpan ke file .tflite
with open("model_klasifikasi.tflite", "wb") as f:
    f.write(tflite_model)

print("Model berhasil dikonversi ke TFLite!")

Epoch 1/200


c:\laragon\bin\python\python-3.10.11\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0738 - loss: 3.5139 - val_accuracy: 0.2819 - val_loss: 3.0440
Epoch 2/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2040 - loss: 2.9605 - val_accuracy: 0.4093 - val_loss: 2.5234
Epoch 3/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3910 - loss: 2.4122 - val_accuracy: 0.5521 - val_loss: 1.9830
Epoch 4/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4259 - loss: 2.0399 - val_accuracy: 0.6293 - val_loss: 1.5410
Epoch 5/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5308 - loss: 1.6835 - val_accuracy: 0.7066 - val_loss: 1.2160
Epoch 6/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5914 - loss: 1.4113 - val_accuracy: 0.7490 - val_loss: 0.9896
Epoch 7/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6893 - loss: 1.1387 - val_accuracy: 0.7413 - val_loss: 0.8280
Epoch 8/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7041 - loss: 1.0301 - val_accuracy: 0.7761 - val_loss: 0.

Akurasi: 94.44%
INFO:tensorflow:Assets written to: C:\Users\Aldan\AppData\Local\Temp\tmpgeshssuc\assets


INFO:tensorflow:Assets written to: C:\Users\Aldan\AppData\Local\Temp\tmpgeshssuc\assets


Saved artifact at 'C:\Users\Aldan\AppData\Local\Temp\tmpgeshssuc'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 60), dtype=tf.float32, name='keras_tensor_6')
Output Type:
  TensorSpec(shape=(None, 36), dtype=tf.float32, name=None)
Captures:
  1891697955712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1890431386192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1890431397456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1890431388304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1890431400448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1890431520432: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model berhasil dikonversi ke TFLite!


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

import joblib

# Simpan scaler


# 1. Load Data
df = pd.read_csv(r"C:\Users\Aldan\Desktop\ASLClassification\Code\hand_features.csv")  # Ganti dengan nama file kamu

# 2. Pisahkan fitur dan label
X = df.drop("label", axis=1).values
y = df["label"].values



# encoder = LabelEncoder()
# y_encoded = encoder.fit_transform(y)
# y_categorical = to_categorical(y_encoded, num_classes=36)


36


In [13]:
%pip install pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install mediapipe

  Using cached mediapipe-0.10.21-cp310-cp310-win_amd64.whl.metadata (10 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached jax-0.6.2-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.6.2-cp310-cp310-win_amd64.whl.metadata (1.4 kB)
  Using cached matplotlib-3.10.3-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.8-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached sounddevice-0.5.2-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached sentencepiece-0.2.0-cp310-cp310-win_amd64.whl.metadata (8.3 kB)
  Using cached cffi-1.17.1-cp310-cp310-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached ml_dtypes-0.5.1-cp310-cp310-win_amd

  You can safely remove it manually.
